In [41]:
import numpy as np
import random
from MicroModelNode import MicroModelNode
from MicroModelNetwork import MicroModelNetwork

In [29]:
class BasicNode(MicroModelNode):
    def __init__(self,size_of_input_vector):
        n = size_of_input_vector
        middle_input=.5
        self.weights = [(random.random()-.5)*2 for _ in range(n)]
        self.activation_weights = [(random.random()-.5)*2 for _ in range(n)]

        self.ave_score = 0   
        self.actsum = 0  
        self.middle_input = middle_input  

    def get(self,v):
        value = self.getValue(v)
        activation = self.getActivation(v)
        return( (value,activation) )

    def getValue(self,v):
        total = 0
        for i in range(len(v)):
            total+=(v[i]-self.middle_input)*self.weights[i]
        total = total/len(self.weights)
        total = (total+1)/2
        
        return( total )

    def getActivation(self,v):
        activation_total = 0
        for i in range(len(v)):
            activation_total+=(v[i]-self.middle_input)*self.activation_weights[i]
        activation_total = activation_total/len(self.weights)

        if activation_total > 0 or activation_total <= .5:
            activation_total =  abs(abs(activation_total - .25)-.25)*2 
        else:
            activation_total = 0
        
        return( activation_total )

    def evaluate(self,v,expected):
        result = self.getValue(v)
        activation = self.getActivation(v)
        score = 1-abs(result-expected)
        adjusted_score = score*activation
        if (self.actsum+activation) != 0:
            self.ave_score = (self.ave_score*(self.actsum)+adjusted_score) / (self.actsum+activation)
        self.actsum += activation

    def clear_evaluation(self):
        self.ave_score = 0
        self.actsum = 0

    def getWeights(self):
        return(self.weights)

    def getActivationWeights(self):
        return(self.activation_weights)

    def getAverageScore(self):
        return(self.ave_score)

In [35]:
class MicroModelNetwork2(MicroModelNetwork):

    def __init__(self,vectorsize):
        num_nodes = 200
        self.vectorsize = vectorsize
        self.nodes = [BasicNode(vectorsize) for _ in range(num_nodes)]
        self.max_output = None
        self.min_output = None

        # for sorting
        self.unsortedNodes = None
        self.sortedNodes = None
        self.average_vweights = None
        self.average_aweights = None
    
    def get(self,v):
        average = 0
        total_activation = 0
        for i in range(len(self.nodes)):
            local_activation = self.nodes[i].getActivation(v)
            local_result = self.nodes[i].getValue(v)
            average += local_result * local_activation
            total_activation += local_activation
        if total_activation != 0:
            average = average / total_activation
        else:
            average = 0
        return(average)
    
    def getScaled(self,v):
        unscaled = self.get(v)
        if self.max_output!=0 or self.min_output!=0:
            diff = abs(self.max_output - self.min_output)
            average = (unscaled-self.min_output)/diff
        return(average)
    
    def getScaled2(self,v):
        scaled = self.getScaled(v)
        if scaled < 0:
            return(0)
        elif scaled > 1:
            return(1)
        else:
            return(scaled)
    
    def scale(self,v):
        unscaled = self.get(v)
        if self.max_output == None:
            self.max_output = unscaled
        else:
            self.max_output = max( [self.max_output,unscaled] ) 
        
        if self.min_output == None:
            self.min_output = unscaled
        else:
            self.min_output = min( [self.min_output,unscaled] ) 

    def evaluate(self, v, expected):
        for i in range(len(self.nodes)):
            self.nodes[i].evaluate(v,expected)
        self.scale(v)
            
    def __prune(self,remove_indices):
        self.nodes = [self.nodes[i] for i in range(len(self.nodes)) if i not in remove_indices]

    def __createADerivativeNode(self):
        newnode = BasicNode(self.vectorsize)
        dweights = [random.random() for _ in range(len(self.nodes))]
        new_Weights = np.array([0 for _ in range(self.vectorsize)])

        for i,node in enumerate(self.nodes):
            new_Weights = new_Weights + np.array(node.weights) * dweights[i]
        new_Weights = new_Weights/sum(dweights)
        new_Weights = list(new_Weights)
        newnode.weights = new_Weights
        return(newnode)

    def addDerivativeNodes(self,n):
        newnodes = [self.__createADerivativeNode() for _ in range(n)]
        self.nodes = self.nodes + newnodes
     
    def addNodes(self,n,MicroModelNodeClass=BasicNode):
        newnodes = [MicroModelNodeClass(self.vectorsize) for _ in range(n)]
        self.nodes = self.nodes+newnodes

    def pruneLowestN(self,n):
        x = [ (self.nodes[i].ave_score,i) for i in range(len(self.nodes)) ]
        x.sort(key=lambda x:x[0])
        x = x[:n]
        remove_indices = [val[1] for val in x]
        self.__prune(remove_indices)

    def pruneAveScore(self,thresh):
        remove_indices = set()
        for i in range(len(self.nodes)):
            if self.nodes[i].ave_score < thresh:
                remove_indices.add(i)
        self.__prune(remove_indices)

    def pruneLowest(self,thresh,n):
        x = [ (self.nodes[i].ave_score,i) for i in range(len(self.nodes)) ]
        x.sort(key=lambda x:x[0])
        x = x[:n]
        remove_indices = [val[1] for val in x if val[0] < thresh]
        self.__prune(remove_indices)
        return( len(remove_indices) )
    
    def pruneRandom(self,n):
        remove_indices = set()
        for _ in range(n):
            remove_indices = set([random.randint(0,len(self.nodes)-1)])
            self.__prune(remove_indices)
        return( len(remove_indices) )

    def clearRange(self):
        self.max_output     = None
        self.min_output     = None

    def __cosine_sim(self,v1,v2):
        numerator = 0
        denom_left = 0
        denom_right = 0
        for i in range(len(v1)):
            numerator += v1[i] * v2[i]
            denom_left += pow(v1[i],2)
            denom_right += pow(v2[i],2)
        denom_left = pow(denom_left,.5)
        denom_right = pow(denom_right,.5)
        denominator = denom_left*denom_right
        if denominator == 0:
            return(0)
        answer = numerator / denominator
        return(answer)

    def __node_sim(self,node1, node2):
        v1 = node1.weights + node1.activation_weights
        v2 = node2.weights + node2.activation_weights
        answer = self.__cosine_sim(v1,v2)
        return(answer)

    def remove_duplicates(self,sim_thresh,maximum_to_remove=None):
        nonduplicates = self.nodes.copy()
        removed = 0
        maximum_met = False
        for i,__ in enumerate(nonduplicates,start=0):
            if i >= len(nonduplicates):
                    maximum_met = True
                    break
            node = nonduplicates[i]
            if maximum_met:
                break
            for j, ___ in enumerate(nonduplicates,start=i+1):
                if j >= len(nonduplicates):
                    maximum_met = True
                    break
                othernode = nonduplicates[j]
                similarity = self.__node_sim(node,othernode)
                if similarity >= sim_thresh:
                    nonduplicates = nonduplicates[:j] + nonduplicates[j+1:]
                    removed += 1
                if maximum_to_remove!=None and removed >= maximum_to_remove:
                    maximum_met = True
                    break
        self.nodes = nonduplicates
        return(removed)        

In [ ]:
class ComplexFunction:
    def __init__(self, input_len,number_of_xors=5):
        self.random_xors = [random.randint(0,input_len-1) for _ in range(number_of_xors)]
    
    def doRandomXors(self,v):
        output = None
        for num in self.random_xors:
            if output == None:
                output = round(v[num])
            else:
                output ^= round(v[num])
        return(output)
    
    def getResult(self,v):
        return(self.doRandomXors(v))


In [38]:
test = MicroModelNetwork2(1)
#cf = ComplexFunction(5,number_of_xors=10)

In [932]:
cf.random_xors

[4, 3, 0, 4, 3, 2, 3, 0, 3, 1]

In [39]:

for i in range(0,10):
    #print(i,end=" ")
    test.addDerivativeNodes(60)
    test.addNodes(60)
    test.addDerivativeNodes(80)
    for _ in range(160):
        #v = [random.choice([1,0]) for _ in range(5)]
        #expected = cf.getResult(v)
        v = [.5]
        expected = 1
        test.evaluate(v,expected)
        v = [0]
        expected = 0
        test.evaluate(v,expected)
        v = [1]
        expected = 0
        test.evaluate(v,expected)


  
    duplicates = test.remove_duplicates(sim_thresh=.97,maximum_to_remove=0)
    #print(f"duplicates: {duplicates}")
    lowest = test.pruneLowest(.99,200-duplicates)
    #print(f"lowest: {lowest}")
    rand = test.pruneRandom(200-lowest-duplicates)
    #print(f"random: {rand}")
    #test.clearRange()


In [1117]:
v = [random.choice([1,0]) for _ in range(5)]
expected = cf.getResult(v)
test.scale(v)
actual = test.getScaled2(v)
print(v)
print(f"expected: {expected}")
print(f"actual: {actual}")

[0, 0, 1, 0, 1]
expected: 1
actual: 0.6413675977297508


In [63]:
test.pruneLowest(.99,5)

5

In [91]:
test.nodes[4].get([.5])

(0.5, 0.0)

In [67]:
print(test.getScaled([1]))
#print(test.getScaled([0,1]))
#print(test.getScaled([1,1]))
#print(test.getScaled([0,0]))

0.5195437101441681


In [303]:
0.4214748683099201 / abs(test.min_output - test.max_output)

1.5017948923088322

In [304]:
test.min_output

0.37035963922697135

In [158]:
test.getScaled([0,1])

-0.08658902924624574

In [248]:
test.clearRange()

In [196]:
test.scale([1,0])
test.scale([0,1])
test.scale([1,1])
test.scale([0,0])

In [202]:
round(.5)

0

In [381]:
cf.random_xors

[]